In [ ]:
!obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -osdf > LIG.sdf
!obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -opdb > LIG.pdb

In [ ]:
import mdtraj as md
from Bio.PDB import PDBList

pdbl = PDBList()
pdbl.download_pdb_files(['3vte'], pdir='./', file_format='pdb')

# # Extract components and save with mdtraj
trj3VTE = md.load_pdb('pdb3vte.ent')

# Holo-enzyme
trj3VTE_holo = trj3VTE.atom_slice(
    trj3VTE.top.select('not water'))
trj3VTE_holo.save_pdb('3vte_holo.pdb')

# Apo-form
trj3VTE_apo = trj3VTE.atom_slice(
    trj3VTE.top.select('protein'))
trj3VTE_apo.save_pdb('3vte_apo.pdb')

# FAD extracted
trjFAD = trj3VTE.atom_slice(
    trj3VTE.top.select('resname FAD'))
trjFAD.save_pdb('FAD_ext.pdb')   

# # NAG extracted
# trjNAG = trj3VTE.atom_slice(
#     trj3VTE.top.select('resname NAG'))
# trjNAG.save_pdb('NAG_ext.pdb')    

In [ ]:
from pdbfixer import *
from simtk.openmm.app import PDBFile

fixer = PDBFixer('3vte_apo.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(5.2)

with open('3vte_apo_fxd.pdb', 'w+') as outfile:
    PDBFile.writeFile(
        fixer.topology, fixer.positions, outfile)

In [ ]:
%%capture
!obabel -ipdb FAD_ext.pdb -osdf > FAD_ext.sdf
# !obabel -ipdb NAG_ext.pdb -osdf > NAG_ext.sdf

In [ ]:
import parmed as pmd

from simtk import openmm, unit
from simtk.openmm import app, LangevinIntegrator
from simtk.openmm.app import PDBFile, NoCutoff, HBonds

from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator
from openforcefield.typing.engines.smirnoff import ForceField

# # Load Amber force fields
omm3VTE_ff = app.ForceField('Amber14-all.xml')
# omm3VTE_ff.registerTemplatePatch(residue, patch, ...)

# Parameterize apo-protein
omm3VTE = PDBFile('3vte_apo_fxd.pdb')
receptor_system = omm3VTE_ff.createSystem(omm3VTE.topology)
pmd3VTE = pmd.openmm.load_topology(
    omm3VTE.topology,
    receptor_system,
    xyz=omm3VTE.positions)

# # # Parameterize NAG glycosylations
# ommNAG = PDBFile('NAG_ext.pdb')
# glycosyl_system = mm3VTE_ff.createSystem(ommNAG.topology)
# pmdNAG = pmd.openmm.load_topology(
#     ommNAG.topology,
#     glycosyl_system,
#     xyz=ommNAG.positions)

# # Load Parsley force field
offLIG_ff = ForceField('openff_unconstrained-1.2.1.offxml')

# # Parameterize FAD cofactor
# ommFAD = PDBFile('FAD_ext.pdb')
# molFAD = Molecule('FAD_ext.sdf')
# cofactor_system = offLIG_ff.create_openmm_system(
#     molFAD.to_topology())
# pmdFAD = pmd.openmm.load_topology(
#     ommFAD.topology,
#     cofactor_system,
#     xyz=ommFAD.positions)

# Parameterize CBGA substrate
ommCBGA = PDBFile('LIG.pdb')
molCBGA = Molecule('LIG.sdf')
ligand_system = offLIG_ff.create_openmm_system(
    molCBGA.to_topology())
pmdCBGA = pmd.openmm.load_topology(
    ommCBGA.topology,
    ligand_system,
    xyz=ommCBGA.positions)

In [ ]:
# # Assemble holo-enzyme with docked substrate
# pmdCPLX = pmd3VTE + pmdMNAG + pmdFAD + pmdCBGA 
# pmdCPLX = pmd3VTE + pmdFAD + pmdCBGA
pmdCPLX = pmd3VTE + pmdCBGA 
pmdCPLX.visualize()

In [ ]:
# # Convert structure to OpenMM
# complex_system = pmdCPLX.createSystem(
#     nonbondedMethod=NoCutoff,
#     nonbondedCutoff=9.0*unit.angstrom,
#     constraints=HBonds)

In [ ]:
# CHARMM-GUI Glycan modeller 
# # Job ID: 0147660438
CGUI_JOBID = '0147660438'
CGUI_SLUG = f'../structures/jobid-{CGUI_JOBID}/'  

pmd.charmm.CharmmPsfFile
    .load_parameters(CharmmParameterSet)
    .prune_empty_terms()
    .to_dataframe()
    .omm_set_virtual_sites()
    .createSystem()
pmd.charmm.CharmmCrdFile

system = psf.createSystem(
    params=params, nonbondedMethod=NoCutoff,
    nonbondedCutoff=1*nanometer, constraints=HBonds)
simulation = Simulation(model.topology, system, integrator)
simulation.context.setPositions(model.positions)


In [ ]:
# Load holo-enzyme
# Load CHARMM files
FADPAR = CGUI_SLUG + 'fad/'
TOPPAR = CGUI_SLUG + toppar/'

# # Unicode error
# hhoRST_pat = 'em.water.rst'
# with open(hhoRST_path, 'r', encoding='ISO-8859-1') as wfile:
#     app.charmmcrdfiles.CharmmRstFile(wfile.read())

psf = CharmmPsfFile(CGUI_SLUG + 'step1_pdbreader.psf')
pdb = PDBFile(CGUI_SLUG + 'step1_pdbreader.pdb')

parset = [TOPPAR + f for f in os.listdir(TOPPAR)] + [FADPAR + f for f in ['fad.prm', 'fad_g.rtf', 'fad.rtf', 'ndihe.str']]
params = CharmmParameterSet(*parset)
psf.loadParameters(params)

In [ ]:
# Minimization
integrator = LangevinIntegrator(
    300*unit.kelvin, 
    1/unit.picosecond, 
    0.002*unit.picoseconds)

simulation = app.Simulation(pmdCPLX.topology, complex_system, integrator)
simulation.context.setPositions(pmdCPLX.positions)
simulation.minimizeEnergy()